In [1]:
import pandas as pd
import numpy as np
from model_tuner import *

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

In [2]:
from sklearn.datasets import fetch_california_housing
import sklearn
sklearn.__version__

'1.4.1.post1'

In [ ]:
# # # Generate a synthetic dataset
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, random_state=3)

In [3]:
# Generate a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, random_state=3)

# Specify the fraction of missing values
missing_rate = 0.1 # 10% of the data

# Introduce missing values
n_missing_samples = int(np.floor(X.shape[0] * X.shape[1] * missing_rate))
missing_features = np.random.randint(0, X.shape[1], n_missing_samples)
missing_samples = np.random.randint(0, X.shape[0], n_missing_samples)

X[missing_samples, missing_features] = np.nan

# Now X contains approximately 10% missing values randomly distributed across features


In [4]:
rf_makeclass = RandomForestClassifier(random_state=3)

estimator_name = "rf_makeclass"

In [ ]:
# # Define the hyperparameters for Random Forest
# rf_n_estimators = [100, 200, 300]
# rf_max_depths = [None, 5, 10]
# rf_criterions = ['gini', 'entropy']

# rf_parameters = [{'RF__n_estimators': [n_estimator], 'RF__max_depth': [max_depth], 
#                   'RF__criterion': [criterion]}
#                  for n_estimator in rf_n_estimators
#                  for max_depth in rf_max_depths
#                  for criterion in rf_criterions]


In [5]:
lr = LogisticRegression(class_weight="balanced", C=1, max_iter=1000)

estimator_name = "lr"
# Set the parameters by cross-validation
tuned_parameters = [{estimator_name + "__C": np.logspace(-4, 0, 10)}]

In [6]:
kfold = False
calibrate = True

model = Model(
    name="Random Logistic Regression",
    estimator_name=estimator_name,
    calibrate=calibrate,
    estimator=lr,
    kfold=kfold,
    # impute=True,
    # scaler_type="min_max_scaler",
    stratify=False,
    grid=tuned_parameters,
    randomized_grid=False,
    n_iter=3,
    scoring=["roc_auc", "precision_macro"],
    # n_splits=2,
    random_state=3,
)

model.grid_search_param_tuning(X, y)

model.fit(X, y)

y: [0 1 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 0
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 0
 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0
 1 1 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1
 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0
 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1
 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0
 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1
 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 1 1 0 0

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# # Load the California housing dataset
# california_housing = fetch_california_housing(as_frame=True)

In [ ]:
# df = california_housing["frame"]

In [ ]:
iris = load_iris()
iris = pd.DataFrame(
    data=np.c_[iris["data"], iris["target"]],
    columns=iris["feature_names"] + ["target"],
)
features = [col for col in iris.columns if col != "target"]
target = "target"

X = iris[features].values  # independant variables
y = iris[target].values.astype(int)  # dependent variable

# breast_sk = load_breast_cancer()
# breast = pd.DataFrame(
#     data=np.c_[breast_sk.data, breast_sk.target],
# )
# breast.columns = list(breast_sk.feature_names) + ["target"]
# features = [col for col in breast.columns if col != "target"]
# target = "target"

# X = breast[features].values  # independant variables
# y = breast[target].values.astype(int)  # dependent variable

lr = LogisticRegression(class_weight="balanced", C=1, max_iter=1000)

estimator_name = "lr"
# Set the parameters by cross-validation
tuned_parameters = [{estimator_name + "__C": np.logspace(-4, 0, 10)}]

In [ ]:
kfold = True
calibrate = True

model = Model(
    name="Iris_model",
    estimator_name=estimator_name,
    calibrate=calibrate,
    estimator=lr,
    kfold=kfold,
    stratify=True,
    grid=tuned_parameters,
    randomized_grid=False,
    n_iter=3,
    scoring=["roc_auc_ovr", "precision_macro"],
    n_splits=2,
    random_state=3,
)

model.grid_search_param_tuning(X, y)

model.fit(X, y)

In [ ]:
if model.calibrate:
    model.calibrateModel(X, y)
else:
    pass

if kfold:
    print(model.xval_output["train_score"], model.xval_output["test_score"])
    for i in range(len(model.xval_output["estimator"])):
        print("\n" + str(i) + " Fold: ")
        if calibrate:
            importance = (
                model.xval_output["estimator"][i]
                .calibrated_classifiers_[i]
                .estimator.steps[1][1]
                .coef_[0]
            )
        else:
            importance = model.xval_output["estimator"][i].steps[1][1].coef_[0]

        sort_imp_indx = np.argsort(importance)[::-1]
        # print(importance)
        # print(sort_imp_indx)
        for i in sort_imp_indx:
            print("Feature: %s, Score: %.5f" % (features[i], importance[i]))
else:
    if calibrate:
        importance = model.estimator.estimator.steps[1][1].coef_[0]
    else:
        importance = model.estimator.steps[1][1].coef_[0]
    sort_imp_indx = np.argsort(importance)[::-1]
    # print(importance)
    # print(sort_imp_indx)
    # summarize feature importance
    for i in sort_imp_indx:
        print("Feature: %s, Score: %.5f" % (features[i], importance[i]))
